In [62]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
# import transformers
# import torch

model = "tiiuae/falcon-7b"

tokenizer = AutoTokenizer.from_pretrained(
    model, 
    add_eos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [63]:
dataset = load_dataset(
    "json",
    name='ihop_vector_v1',
    data_files='../datasets/compiled.jsonl', 
    split='train[:]'
).train_test_split(test_size=0.05)

Found cached dataset json (/home/lawrence/.cache/huggingface/datasets/json/ihop_vector_v1-5e733bd2631bd9e1/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


In [57]:
system_msg = "You are a helpful executive assitant. Your Job is to respond to user inputs with helpful and factual answers. The user may pass context, if context is provided use the context combined with your knowledge to provide answers."

def preprocess_dataset(x):
    prompt = f"<|SYSTEM|>{system_msg}<|END_SYSTEM|>"
    if 'context' in x:
        if len(x['context']) > 0:
            prompt += f"<|CONTEXT|>{x['context']}<|CONTEXT|>"
    if 'message' in x:
        prompt += x['message']
    else:
        prompt += f'<|USER_INPUT|>{x["input"]}<|END_USER_INPUT|>'
        prompt += f'<|RESPONSE|>{x["response"]}<|END_RESPONSE|>'
    return {
        **x,
        'prompt': prompt
    }

In [64]:
prompt_ds = dataset.map(preprocess_dataset)
tokenized_ds = prompt_ds.map(
    lambda samples: tokenizer(
        samples['prompt'], 
        padding=True,
        truncation=True,
    ), 
    batched=True
)

In [68]:
tokenizer.eos_token_id

65024

In [ ]:
tokenized_ds.save_to_disk("../dataset/tokenized_ds")

In [66]:
tokenized_ds['train'][4]['input_ids']

[39,
 103,
 25060,
 33923,
 54146,
 1357,
 362,
 241,
 5335,
 7940,
 820,
 25023,
 25,
 2085,
 10999,
 304,
 271,
 3869,
 271,
 2735,
 19345,
 335,
 5335,
 273,
 37342,
 6864,
 25,
 390,
 2735,
 724,
 1285,
 4436,
 23,
 565,
 4436,
 304,
 2930,
 745,
 248,
 4436,
 6725,
 335,
 402,
 2882,
 271,
 1586,
 6864,
 8275,
 103,
 12281,
 74,
 25060,
 33923,
 103,
 4836,
 103,
 57845,
 54146,
 29181,
 299,
 1757,
 271,
 1849,
 267,
 5647,
 271,
 53019,
 23,
 325,
 304,
 520,
 4952,
 335,
 21805,
 273,
 1286,
 19698,
 13313,
 963,
 345,
 847,
 5864,
 19698,
 39,
 103,
 12281,
 74,
 57845,
 103,
 4836,
 103,
 61173,
 1523,
 5019,
 54146,
 18252,
 23,
 360,
 451,
 1035,
 334,
 29135,
 8852,
 25,
 193,
 193,
 4031,
 629,
 304,
 241,
 7462,
 325,
 299,
 418,
 736,
 314,
 5616,
 271,
 248,
 3605,
 299,
 729,
 271,
 23,
 273,
 418,
 441,
 485,
 22008,
 8275,
 103,
 12281,
 74,
 61173,
 1523,
 5019,
 103,
 4836,
 103,
 57845,
 54146,
 3456,
 13713,
 304,
 11590,
 25,
 1265,
 441,
 299,
 2413,
 313,
 70

In [45]:
prompt_ds['train'][:5]

{'input': ['How can I get the best deal on a new car.',
  'What is the best type of guitar for a beginner.',
  'What are the key ingredients for the local dish tortilla españolas.',
  'What are some tips for budgeting finances.',
  'Can you explain the basics of self-driving cars.'],
 'response': ['To get the best deal on a new car, you should shop around and compare prices from multiple dealerships. You should also research any available incentives or specials, like discounts for loyalty programs, student incentives, or military discounts. Additionally, you may be able to get a better deal by selling or trading in your current vehicle. Finally, consider buying a slightly used car for a cheaper price, as you may also get an extended warranty or other benefits.',
  'The best type of guitar for a beginner is an acoustic guitar. It is easier to learn on, and the sound it produces is great for those just starting out.',
  'The key ingredients for the local dish tortilla españolas are potat